In [10]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
import pinecone
from pinecone import Pinecone, ServerlessSpec

In [13]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '3d29d085d90daf45ec65704e9cdb77ba', 'Date': 'Fri, 23 Aug 2024 11:33:29 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [14]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Emily Thornton',
  'subject': 'Biology',
  'stars': 4,
  'review': "Dr. Thornton's lectures are engaging and well-structured. She's always willing to help during office hours."},
 {'professor': 'Prof. Michael Chang',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant instructor! Prof. Chang makes complex topics easy to understand and provides great real-world examples.'},
 {'professor': 'Dr. Samantha Lee',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Dr. Lee knows her subject well, but her lectures can be a bit dry. More interactive sessions would be helpful.'},
 {'professor': 'Prof. Robert Johnson',
  'subject': 'History',
  'stars': 4,
  'review': "Prof. Johnson's passion for history is contagious. His anecdotes make the subject come alive."},
 {'professor': 'Dr. Sarah Martinez',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Dr. Martinez is an excellent teacher. Her lab sessions are well-organized and her explanations are crystal

In [15]:
processed_data = []
client = OpenAI()

for review in data ['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [16]:
processed_data[0]

{'values': [-0.008654156,
  -0.01858891,
  0.036342062,
  0.023428228,
  0.015501992,
  0.016041191,
  -0.009678635,
  0.0156233115,
  -0.033214703,
  -0.005496467,
  -0.0022006088,
  0.017402671,
  0.027539626,
  -0.031219663,
  0.010595274,
  0.045481496,
  -0.026299465,
  -0.024344867,
  0.042408057,
  0.06799308,
  0.03763614,
  0.001855184,
  0.07419388,
  -0.02073223,
  -0.050091654,
  -0.022215027,
  -0.003986708,
  0.0023118188,
  -0.013722633,
  0.028173186,
  0.04165318,
  -0.014437072,
  -0.004131618,
  -0.025571546,
  -0.060767807,
  0.05432437,
  0.008896795,
  0.0080677755,
  0.053434692,
  0.022147628,
  0.022525068,
  -0.015124552,
  -0.024964947,
  0.032244142,
  0.023064267,
  -0.0024432486,
  -0.016769111,
  0.0063187466,
  0.04151838,
  0.025072787,
  -0.047934856,
  0.0021230988,
  0.025436746,
  -0.015906392,
  -0.05979725,
  0.024776228,
  0.022309389,
  0.011896093,
  0.030437823,
  -0.026083786,
  0.045535415,
  -0.016472552,
  -0.01705219,
  0.001032062,
  -0.

In [17]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}